In [1]:
from collections import Counter
import pickle
from keras_preprocessing.sequence import pad_sequences
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras_contrib.layers.crf import CRF

Using TensorFlow backend.
/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/py3.7/lib/python3.7/sit

In [3]:
# 获取词典映射
def getWordAndTagId(filePath):
    with open(filePath, encoding='utf-8') as file:
        wordsAndtags = [line.split(' ') for line in file]

        words = [i[0].strip() for i in wordsAndtags if len(i)==2 ]

        tags = [i[1].strip()  for i in wordsAndtags if len(i)==2 ]
        words = Counter(words)
        tags = Counter(tags)
        words = sorted(words.items(), key=lambda x: -x[1])
        words = words[:4000]
        tags = sorted(tags.items(), key=lambda x: -x[1])
        word_size = len(words)
        word2id = {count[0]: index for index, count in enumerate(words,start=1)}
        id2word = {index: count[0] for index, count in enumerate(words,start=1)}
        tag2id = {count[0]: index for index, count in enumerate(tags)}
        id2tag = {index: count[0] for index, count in enumerate(tags)}
        word2id['<PAD>'] = 0
        word2id['<UNK>'] = word_size + 1
        return word2id, tag2id, id2word, id2tag
word2id, tag2id, id2word, id2tag = getWordAndTagId('train.txt')

def saveWordAndTagId(word2id,tag2id):
    word2idFile = open('data/word2id', 'wb')
    tag2idFile = open('data/tag2id', 'wb')
    pickle.dump(word2id, word2idFile)
    pickle.dump(tag2id, tag2idFile)
    word2idFile.close()
    tag2idFile.close()
#这句被注释掉的代码，是将其保存起来，以便下次使用
#saveWordAndTagId(word2id, tag2id)


def getSentencesAndTags(filePath):
    '''
    从文件里面获取句子和标注
    :param filePath:
    :return:
    '''
    with open(filePath,encoding='utf-8') as file:
        wordsAndtags=[line.split() for line in file]
        sentences=[]
        tags=[]
        sentence=[]
        tag=[]
        for wordAndTag in wordsAndtags:
            if len(wordAndTag)==2:
                sentence.append(wordAndTag[0])
                tag.append(wordAndTag[1])
            else:
                sentences.append(sentence)
                tags.append(tag)
                sentence=[]
                tag = []
    return sentences,tags
# 获取句子和标注
sentences, tags=getSentencesAndTags('train.txt')

In [4]:
def sentencesAndTags2id(sentences,tags,word2id, tag2id):
    '''
    将句子和标注转换为id
    :param sentences:
    :param tags:
    :param word2id:
    :param tag2id:
    :return:
    '''
    sentencesIds = [[word2id.get(char,len(word2id)) for char in sentence] for sentence in sentences]
    tagsIds = [[tag2id[char] for char in tag] for tag in tags]
    return sentencesIds,tagsIds
# 将句子和标注转换为id
sentencesIds, tagsIds = sentencesAndTags2id(sentences, tags,word2id, tag2id)


In [5]:
# 将句子和标注进行填充，确保输入维度一致
sentencesIds = pad_sequences(sentencesIds, padding='post')
tagsIds = pad_sequences(tagsIds, padding='post')
print(sentencesIds.shape)
print(tagsIds.shape)

(70, 91)
(70, 91)


In [6]:



def loadWordAndTagId():
    word2idFile = open('data/word2id', 'wb')
    tag2idFile = open('data/tag2id', 'wb')
    word2id = pickle.load(word2idFile)
    tag2id = pickle.load(tag2idFile)
    return word2id, tag2id

def model(vocabSize,embeddingDim,inputLength,tagSize):
    model = Sequential()
    model.add(Embedding(vocabSize + 1,embeddingDim,input_length=inputLength,trainable=False,mask_zero=True))
    model.add(Bidirectional(LSTM(50, return_sequences=True)))
    model.add(TimeDistributed(Dense(tagSize)))
    crf_layer = CRF(tagSize, sparse_target=True)
    model.add(crf_layer)
    model.compile('adam', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])
    model.summary()
    return model


def get_triple(y_labels,input_data):
    subjects, predicates, objects = '', '', ''
    for s, t in zip(input_data, y_labels):
        if t in ('B-SUBJECT', 'I-SUBJECT'):
            subjects += ' ' + s if (t == 'B-SUBJECT') else s
        if t in ('B-PREDICATE', 'I-PREDICATE'):
            predicates += ' ' + s if (t == 'B-PREDICATE') else s
        if t in ('B-OBJECT', 'I-OBJECT'):
            objects += ' ' + s if (t == 'B-OBJECT') else s
    y_counter = Counter(y_labels)
    # 主 谓 宾、宾..
    if y_counter['B-SUBJECT'] == 1 and y_counter['B-PREDICATE'] == 1 and y_counter['B-OBJECT'] >= 1:
        for object in objects.strip().split(' '):
            print('抽取结果：',subjects+'-'+predicates+'->'+object)
    # 主 谓 宾、谓 宾..
    elif y_counter['B-SUBJECT'] == 1 and y_counter['B-PREDICATE'] > 1 and y_counter['B-OBJECT'] >= 1:
        for i in range(len(predicates.strip().split(' '))):
            print('抽取结果：',subjects+'-'+predicates.strip().split(' ')[i]+'->'+objects.strip().split(' ')[i])
    elif y_counter['B-SUBJECT'] > 1:
        get_triple(y_labels[y_labels.index('I-OBJECT'):],input_data[y_labels.index('I-OBJECT'):])


def predict(model,input_data,length,word2id,id2tag):
    '''
    预测
    :param model:
    :param inputData:
    :param length:
    :param word2id:
    :param id2tag:
    :return:
    '''
    input = [word2id.get(char,len(word2id)) for char in input_data]
    input = np.reshape(input,[1,-1])
    # 填充
    input = pad_sequences(input,maxlen=length)
    y = model.predict(input)
    # 输出为三维  转为二维
    y = y.reshape([-1,7])
    y = np.argmax(y,axis=1)
    # 去除填充的部分
    y = y[len(y)-len(input_data):]
    y = [id2tag[i] for i in y]
    get_triple(y,input_data)
    # print(y)

In [7]:
# 载入模型
model=model(len(word2id),100,sentencesIds.shape[1],len(tag2id))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 91, 100)           60000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 91, 100)           60400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 91, 7)             707       
_________________________________________________________________
crf_1 (CRF)                  (None, 91, 7)             119       
Total params: 121,226
Trainable params: 61,226
Non-trainable params: 60,000
_________________________________________________________________


In [6]:
"""
下面这两行是做700个epoch训练模型并保存，如果只调用的话需要将其注释掉
"""
history = model.fit(sentencesIds, tagsIds.reshape([len(tagsIds),-1,1]), epochs=700)
model.save('kg-mode.model')




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
/root/anaconda3/envs/py3.7/lib/python3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 91, 100)           60000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 91, 100)           60400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 91, 7)             707       
_________________________________________________________________
crf_1 (CRF)                  (None, 91, 7)             119       
Total params: 121,226
Trainable params: 61,226
Non-trainable params: 60,000
_________________________________________________________________

Epoch 1/700


2022-09-27 14:27:54.893786: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-09-27 14:27:54.902943: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399895000 Hz
2022-09-27 14:27:54.906743: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e18fdcc1d0 executing computations on platform Host. Devices:
2022-09-27 14:27:54.906775: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-09-27 14:27:55.337817: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) 

70/70 [==============================] - 3s 49ms/step - loss: 8.5342 - crf_viterbi_accuracy: 0.7464
Epoch 2/700
70/70 [==============================] - 1s 21ms/step - loss: 8.5134 - crf_viterbi_accuracy: 0.7467
Epoch 3/700
70/70 [==============================] - 1s 20ms/step - loss: 8.4898 - crf_viterbi_accuracy: 0.7477
Epoch 4/700
70/70 [==============================] - 1s 20ms/step - loss: 8.4605 - crf_viterbi_accuracy: 0.7508
Epoch 5/700
70/70 [==============================] - 1s 19ms/step - loss: 8.4219 - crf_viterbi_accuracy: 0.7426
Epoch 6/700
70/70 [==============================] - 1s 20ms/step - loss: 8.3695 - crf_viterbi_accuracy: 0.7460
Epoch 7/700
70/70 [==============================] - 1s 19ms/step - loss: 8.2933 - crf_viterbi_accuracy: 0.7461
Epoch 8/700
70/70 [==============================] - 1s 20ms/step - loss: 8.2135 - crf_viterbi_accuracy: 0.7488
Epoch 9/700
70/70 [==============================] - 1s 20ms/step - loss: 8.1937 - crf_viterbi_accuracy: 0.7472
Epoc

In [8]:
from keras_contrib.layers.crf import CRF, crf_loss, crf_viterbi_accuracy
model = load_model('kg-mode.model', custom_objects={"CRF": CRF, 'crf_loss': crf_loss,'crf_viterbi_accuracy': crf_viterbi_accuracy})

2022-09-28 05:27:23.515625: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-09-28 05:27:23.523713: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399895000 Hz
2022-09-28 05:27:23.526880: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55db84dffaa0 executing computations on platform Host. Devices:
2022-09-28 05:27:23.526913: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-09-28 05:27:23.769948: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) 

In [ ]:

while True:
    str = input("原始句子>")
    predict(model,str,sentencesIds.shape[1],word2id,id2tag)

原始句子> 李白和王维同岁，都是孟浩然的好友
